In [212]:
import numpy as np
import pandas as pd
# from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
# import sklearn

In [213]:
import os

---

In [214]:
train  = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')

train = train.drop(['index'],axis = 1)
test =test.drop(['index'],axis = 1)

In [215]:
concat_features = train

mask = concat_features.introelapse > (concat_features.introelapse.quantile(0.75) + 1.5 * (concat_features.introelapse.quantile(0.75) - concat_features.introelapse.quantile(0.25)))
concat_features.introelapse[mask] = concat_features.introelapse.quantile(0.75)

mask = concat_features.testelapse > (concat_features.testelapse.quantile(0.75) + 1.5 * (concat_features.testelapse.quantile(0.75) - concat_features.testelapse.quantile(0.25)))
concat_features.testelapse[mask] = concat_features.testelapse.quantile(0.75)

mask = concat_features.surveyelapse > (concat_features.surveyelapse.quantile(0.75) + 1.5 * (concat_features.surveyelapse.quantile(0.75) - concat_features.surveyelapse.quantile(0.25)))
concat_features.surveyelapse[mask] = concat_features.surveyelapse.quantile(0.75)

train = concat_features



concat_features = test

mask = concat_features.introelapse > (concat_features.introelapse.quantile(0.75) + 1.5 * (concat_features.introelapse.quantile(0.75) - concat_features.introelapse.quantile(0.25)))
concat_features.introelapse[mask] = concat_features.introelapse.quantile(0.75)

mask = concat_features.testelapse > (concat_features.testelapse.quantile(0.75) + 1.5 * (concat_features.testelapse.quantile(0.75) - concat_features.testelapse.quantile(0.25)))
concat_features.testelapse[mask] = concat_features.testelapse.quantile(0.75)

mask = concat_features.surveyelapse > (concat_features.surveyelapse.quantile(0.75) + 1.5 * (concat_features.surveyelapse.quantile(0.75) - concat_features.surveyelapse.quantile(0.25)))
concat_features.surveyelapse[mask] = concat_features.surveyelapse.quantile(0.75)

test = concat_features


In [216]:
train.loc[train['age'] == 0, 'age'] = train['age'].mean()
test.loc[test['age'] == 0, 'age'] = train['age'].mean()

In [217]:
train.loc[train['age'] > 65, 'age'] = 0
test.loc[test['age'] > 65, 'age'] = 0

In [218]:
train.loc[train['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]
test.loc[test['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]

In [219]:
for name in ['education', 'urban', 'gender', 'engnat', 'hand', 'religion',
            'voted', 'married']:

    mergedata = pd.get_dummies(train[name], prefix = name)
    train = pd.concat([train, mergedata], axis=1)
    train = train.drop([name], axis = 1)

In [220]:
for name in ['education', 'urban', 'gender', 'engnat', 'hand', 'religion',
            'voted', 'married']:

    mergedata = pd.get_dummies(test[name], prefix = name)
    test = pd.concat([test, mergedata], axis=1)
    test = test.drop([name], axis = 1)

In [221]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 94 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Q1             14959 non-null  float64
 1   Q2             14931 non-null  float64
 2   Q3             14950 non-null  float64
 3   Q4             14929 non-null  float64
 4   Q5             14962 non-null  float64
 5   Q6             14952 non-null  float64
 6   Q7             14924 non-null  float64
 7   Q8             14952 non-null  float64
 8   Q9             14944 non-null  float64
 9   Q10            14928 non-null  float64
 10  Q11            14941 non-null  float64
 11  Q12            14933 non-null  float64
 12  Q13            14960 non-null  float64
 13  Q14            14964 non-null  float64
 14  Q15            14955 non-null  float64
 15  Q16            14967 non-null  float64
 16  Q17            14963 non-null  float64
 17  Q18            14937 non-null  float64
 18  Q19   

In [222]:
from sklearn.preprocessing import LabelEncoder
ex = LabelEncoder()

con8 = train.country.value_counts().keys()[:8]

for i in range(train.count().max()):
    if(train.country.values[i] not in con8):
        train.country.values[i] = "X"

for i in range(test.count().max()):
    if(test.country.values[i] not in con8):
        test.country.values[i] = "X"

train.country = ex.fit_transform(train.country)

test.country = ex.fit_transform(test.country)

---

In [223]:
# pip install pycaret

In [224]:
from pycaret.classification import *

model = setup(data = train, target = 'nerdiness',use_gpu=True,session_id=2023, normalize = True, normalize_method = 'zscore',transformation=True,train_size=0.85)

,Description,Value
0,session_id,2023
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(15000, 94)"
5,Missing Values,True
6,Numeric Features,38
7,Categorical Features,55
8,Ordinal Features,False
9,High Cardinality Features,False


In [225]:
rf = create_model('rf', cross_validation = False)
lightgbm = create_model('lightgbm', cross_validation = False)
et = create_model('et', cross_validation = False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7765,0.8762,0.8441,0.7637,0.8019,0.5471,0.5507


In [210]:
tuned_cat = tune_model(cat,  n_iter = 50)
tuned_xgb = tune_model(xgb,  n_iter = 50)
tuned_gbr = tune_model(gbr,  n_iter = 50)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7582,0.8248,0.8214,0.7630,0.7911,0.5051,0.5070
1,0.7484,0.8341,0.8105,0.7563,0.7825,0.4851,0.4867
2,0.7564,0.8255,0.8344,0.7550,0.7927,0.4993,0.5029
3,0.7111,0.7856,0.8217,0.7078,0.7605,0.4016,0.4087
4,0.7262,0.7891,0.8025,0.7326,0.7660,0.4378,0.4405
5,0.7404,0.8067,0.8296,0.7380,0.7811,0.4650,0.4698
6,0.7413,0.8148,0.8089,0.7482,0.7774,0.4699,0.4719
7,0.7209,0.7819,0.8025,0.7262,0.7625,0.4262,0.4293
8,0.7316,0.7927,0.8041,0.7383,0.7698,0.4493,0.4516


In [211]:
blended = blend_models(estimator_list = [tuned_cat,tuned_xgb,tuned_gbr], fold = 10, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7813,0.8468,0.8325,0.7873,0.8093,0.5535,0.5547
1,0.7876,0.8643,0.8296,0.7979,0.8134,0.5670,0.5676
2,0.7884,0.8554,0.8392,0.7937,0.8158,0.5678,0.5690
3,0.7307,0.8110,0.8185,0.7312,0.7724,0.4452,0.4495
4,0.7707,0.8264,0.8328,0.7737,0.8021,0.5303,0.5323
5,0.7636,0.8367,0.8360,0.7631,0.7979,0.5145,0.5176
6,0.7671,0.8389,0.8217,0.7748,0.7975,0.5240,0.5253
7,0.7644,0.8147,0.8105,0.7771,0.7935,0.5197,0.5203
8,0.7600,0.8241,0.8137,0.7696,0.7910,0.5097,0.5108


In [226]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7708,0.8369,0.8250,0.7654,0.7941,0.5364,0.5383


In [227]:
final_model = finalize_model(blended)

[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 

In [228]:
predictions = predict_model(final_model, data = test)

In [229]:
predictions['Score']

0        0.5579
1        0.8543
2        0.8861
3        0.6734
4        0.8182
          ...  
35447    0.8260
35448    0.8518
35449    0.9459
35450    0.6126
35451    0.8069
Name: Score, Length: 35452, dtype: float64

In [230]:
predictions['Label']

0        0
1        1
2        1
3        1
4        1
        ..
35447    1
35448    1
35449    1
35450    0
35451    1
Name: Label, Length: 35452, dtype: int32

In [231]:
predictions.loc[predictions['Label'] == 0, 'Score'] = 1 - predictions.loc[predictions['Label'] == 0, 'Score']

In [232]:
predictions['Score']

0        0.4421
1        0.8543
2        0.8861
3        0.6734
4        0.8182
          ...  
35447    0.8260
35448    0.8518
35449    0.9459
35450    0.3874
35451    0.8069
Name: Score, Length: 35452, dtype: float64

In [233]:
sub = predictions['Score']

In [234]:
sub

0        0.4421
1        0.8543
2        0.8861
3        0.6734
4        0.8182
          ...  
35447    0.8260
35448    0.8518
35449    0.9459
35450    0.3874
35451    0.8069
Name: Score, Length: 35452, dtype: float64

In [235]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['nerdiness'] = sub

In [236]:
submission

,index,nerdiness
0,0,0.4421
1,1,0.8543
2,2,0.8861
3,3,0.6734
4,4,0.8182
...,...,...
35447,35447,0.8260
35448,35448,0.8518
35449,35449,0.9459
35450,35450,0.3874


In [237]:
submission.to_csv("./submit/submit10.csv", index = False)